<a href="https://colab.research.google.com/github/eric-castillo05/emotions-wheel-nlp/blob/main/ELMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import polars as pl
from datasets import load_dataset
import pickle
import numpy as np

In [3]:
splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pl.read_parquet('hf://datasets/Jsevisal/go_emotions_wheel/' + splits['train'])

In [4]:
df.head()

text,labels,id
str,list[i64],str
"""My favourite food is anything …",[5],"""eebbqej"""
"""Now if he does off himself, ev…",[5],"""ed00q6i"""
"""WHY THE FUCK IS BAYLESS ISOING""",[7],"""eezlygj"""
"""To make her feel threatened""",[2],"""ed7ypvh"""
"""Dirty Southern Wankers""",[7],"""ed0bdzj"""


In [5]:
df.sample(10)

text,labels,id
str,list[i64],str
"""Which is odd, he was terrible …",[2],"""edlqhtk"""
"""I'm not looking forward to min…",[0],"""ef6wsqa"""
"""While I agree that the biggest…",[1],"""edpm3ul"""
"""Why would we sign [NAME] when …","[7, 3]","""eevg7wz"""
"""So I've just read your whole p…","[0, 3]","""ef8o0xv"""
"""No, though they've been critic…",[5],"""ee5c2b7"""
"""Thanks,keep us posted if you w…",[0],"""eerjiga"""
"""Me too and you made me feel be…",[0],"""eet5spl"""
"""I am really disappointed in ev…",[4],"""ee7cnea"""


In [6]:
%pip install spacy
%pip install spacy_cleaner

In [7]:
import spacy
import spacy_cleaner
from spacy_cleaner.processing import removers, replacers, mutators

In [8]:
!python3 -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 529.0 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
nlp = spacy.load("en_core_web_lg")

In [10]:
docs = df.select('text').to_series().to_list()

In [11]:
docs[:10]

["My favourite food is anything I didn't have to cook myself.",
 'Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead',
 'WHY THE FUCK IS BAYLESS ISOING',
 'To make her feel threatened',
 'Dirty Southern Wankers',
 "OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe PlAyOfFs! Dumbass Broncos fans circa December 2015.",
 'Yes I heard abt the f bombs! That has to be why. Thanks for your reply:) until then hubby and I will anxiously wait 😝',
 'We need more boards and to create a bit more space for [NAME]. Then we’ll be good.',
 'Damn youtube and outrage drama is super lucrative for reddit',
 'It might be linked to the trust factor of your friend.']

In [12]:
from spacy_cleaner import processing, Cleaner

pipeline = spacy_cleaner.Cleaner(
    nlp,
    processing.replace_number_token,
    processing.replace_punctuation_token,
    processing.replace_stopword_token
)

In [13]:
def clean(text, nlp):
    doc = nlp(text.lower().strip())
    tokens = []
    for token in doc:
          tokens.append(token.text)
    return ' '.join(tokens)

In [ ]:
docs = df.select('text').to_series().to_list()
r = [clean(doc, nlp) for doc in docs]

In [202]:
df = df.with_columns(
    pl.Series('clean-text', r).alias('clean-text')
)

In [203]:
df.head()

text,labels,id,clean-text
str,list[i64],str,str
"""My favourite food is anything …",[5],"""eebbqej""","""my favourite food is anything …"
"""Now if he does off himself, ev…",[5],"""ed00q6i""","""now if he does off himself , e…"
"""WHY THE FUCK IS BAYLESS ISOING""",[7],"""eezlygj""","""why the fuck is bayless isoing"""
"""To make her feel threatened""",[2],"""ed7ypvh""","""to make her feel threatened"""
"""Dirty Southern Wankers""",[7],"""ed0bdzj""","""dirty southern wankers"""


In [204]:
with open('train-clean.pkl', 'wb') as f:
  pickle.dump(df, f)

In [15]:
with open('train-clean.pkl', 'rb') as f:
  df_train = pickle.load(f)

In [205]:
df_test = pl.from_dicts(ds['test'][:])

In [206]:
docs = df_test.select('text').to_series().to_list()

In [207]:
docs[:10]

['I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!',
 "It's wonderful because it's awful. At not with.",
 'Kings fan here, good luck to you guys! Will be an interesting game to watch! ',
 "I didn't know that, thank you for teaching me something today!",
 'They got bored from haunting earth for thousands of years and ultimately moved on to the afterlife.',
 'Thank you for asking questions and recognizing that there may be things that you don’t know or understand about police tactics. Seriously. Thank you.',
 'You’re welcome',
 '100%! Congrats on your job too!',
 'I’m sorry to hear that friend :(. It’s for the best most likely if she didn’t accept you for who you are',
 'Girlfriend weak as well, that jump was pathetic.']

In [208]:
r = [clean(doc, nlp) for doc in docs]

In [209]:
df_test = df_test.with_columns(
    pl.Series('clean-text', r).alias('clean-text')
)

In [210]:
df_test.head()

text,labels,id,clean-text
str,list[i64],str,str
"""I’m really sorry about your si…",[4],"""eecwqtt""","""i ’m really sorry about your s…"
"""It's wonderful because it's aw…",[0],"""ed5f85d""","""it 's wonderful because it 's …"
"""Kings fan here, good luck to y…",[0],"""een27c3""","""kings fan here , good luck to …"
"""I didn't know that, thank you …",[0],"""eelgwd1""","""i did n't know that , thank yo…"
"""They got bored from haunting e…",[5],"""eem5uti""","""they got bored from haunting e…"


In [211]:
with open('test-clean.pkl', 'wb') as f:
  pickle.dump(df_test, f)

In [43]:
with open('train-clean.pkl', 'rb') as f:
  df_train = pickle.load(f)

In [16]:
import tensorflow_hub as hub
import tensorflow as tf

In [17]:
model = "https://tfhub.dev/google/elmo/3"
elmo = hub.load(model).signatures['default']

In [216]:
x = ["Roasted ants are a popular snack in Columbia"]

In [10]:
embeddings = elmo(tf.constant(x))["elmo"]

In [13]:
embeddings.shape

TensorShape([1, 8, 1024])

In [18]:
def elmo_vectors(x):
  embeddings = elmo(tf.constant(list(x)))["elmo"]
  return tf.reduce_mean(embeddings, axis = 1)

In [217]:
elmo_vectors(x)

<tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
array([[-0.08238705,  0.07898237,  0.08993176, ..., -0.22516635,
         0.0555426 ,  0.29672703]], dtype=float32)>

In [19]:
list_train = [df_train[i:i+100] for i in range(0, df_train.shape[0], 100)]

In [20]:
list_train

[shape: (100, 4)
 ┌─────────────────────────────────┬───────────┬─────────┬─────────────────────────────────┐
 │ text                            ┆ labels    ┆ id      ┆ clean-text                      │
 │ ---                             ┆ ---       ┆ ---     ┆ ---                             │
 │ str                             ┆ list[i64] ┆ str     ┆ str                             │
 ╞═════════════════════════════════╪═══════════╪═════════╪═════════════════════════════════╡
 │ My favourite food is anything … ┆ [5]       ┆ eebbqej ┆ my favourite food is anything … │
 │ Now if he does off himself, ev… ┆ [5]       ┆ ed00q6i ┆ now if he does off himself , e… │
 │ WHY THE FUCK IS BAYLESS ISOING  ┆ [7]       ┆ eezlygj ┆ why the fuck is bayless isoing  │
 │ To make her feel threatened     ┆ [2]       ┆ ed7ypvh ┆ to make her feel threatened     │
 │ Dirty Southern Wankers          ┆ [7]       ┆ ed0bdzj ┆ dirty southern wankers          │
 │ …                               ┆ …         ┆ …   

In [ ]:
elmo_train = [elmo_vectors(x['clean-text']) for x in list_train]

In [ ]:
with open ('elmo-train.pkl', 'wb') as f:
  pickle.dump(elmo_train, f)

In [34]:
with open ('elmo-train.pkl', 'rb') as f:
  elmo_train = pickle.load(f)

In [23]:
with open ('test-clean.pkl', 'rb') as f:
  df_test = pickle.load(f)

In [24]:
df_test.head()

text,labels,id,clean-text
str,list[i64],str,str
"""I’m really sorry about your si…",[4],"""eecwqtt""","""I ’m really sorry about your s…"
"""It's wonderful because it's aw…",[0],"""ed5f85d""","""it be wonderful because it be …"
"""Kings fan here, good luck to y…",[0],"""een27c3""","""kings fan here good luck to yo…"
"""I didn't know that, thank you …",[0],"""eelgwd1""","""I do not know that thank you f…"
"""They got bored from haunting e…",[5],"""eem5uti""","""they get bored from haunt eart…"


In [25]:
list_test = [df_test[i:i+100] for i in range(0, df_test.shape[0], 100)]

In [31]:
elmo_test = [elmo_vectors(x['clean-text']) for x in list_test]

In [35]:
elmo_train[0]

<tf.Tensor: shape=(100, 1024), dtype=float32, numpy=
array([[-2.0681649e-02, -8.5263856e-02,  1.7512901e-01, ...,
         7.0279561e-02,  9.6149398e-03, -3.8381062e-02],
       [ 3.1486779e-02, -2.1465714e-01,  6.5056026e-02, ...,
        -1.0497018e-01,  2.5125861e-01, -4.5484155e-03],
       [-4.1211601e-02, -7.7113435e-02,  6.7862630e-02, ...,
         1.7554160e-02,  4.5680344e-02, -2.0435799e-04],
       ...,
       [-4.0909234e-03, -2.9983314e-02,  4.9977276e-02, ...,
         1.0161781e-03,  4.1756029e-03,  3.1077968e-02],
       [-1.3440171e-01, -7.2128318e-02,  7.0494756e-02, ...,
        -1.4836657e-01,  2.4608071e-01,  8.2392097e-02],
       [-1.5808007e-02, -4.1046936e-02,  9.1421835e-02, ...,
         2.3020994e-02,  8.2198702e-02,  8.5115461e-03]], dtype=float32)>

In [32]:
elmo_test[0]

<tf.Tensor: shape=(100, 1024), dtype=float32, numpy=
array([[-0.17682931, -0.22035474,  0.01862513, ..., -0.04123634,
         0.2526633 ,  0.09986784],
       [-0.00667305, -0.12619969,  0.08694158, ...,  0.06372865,
         0.09755841,  0.00682761],
       [ 0.11470237, -0.11503435,  0.08863897, ..., -0.08438297,
         0.14478947,  0.0965043 ],
       ...,
       [-0.03638997, -0.03044155,  0.00893117, ...,  0.0151472 ,
         0.05176874,  0.02279315],
       [-0.09297485, -0.02054822,  0.05247753, ..., -0.01451083,
        -0.00449983,  0.01621042],
       [-0.15631011, -0.36106104,  0.09781794, ..., -0.1413809 ,
         0.31204474,  0.05306143]], dtype=float32)>

In [37]:
with open ('elmo-test.pkl', 'wb') as f:
  pickle.dump(elmo_test, f)

In [38]:
from sklearn.model_selection import train_test_split

In [51]:
elmo_train = np.concatenate(elmo_train, axis = 0)

In [53]:
elmo_test = np.concatenate(elmo_test, axis = 0)

In [45]:
df_train.columns

['text', 'labels', 'id', 'clean-text']

In [96]:
labels = df['labels'].list.first().to_numpy()

In [149]:
xtrain, ytrain = elmo_train, df_train['labels'].list.first().to_numpy()
xtest, ytest = elmo_test, df_test['labels'].list.first().to_numpy()

In [150]:
ytrain

array([5, 5, 7, ..., 7, 0, 0])

In [151]:
xtrain

array([[-2.0681649e-02, -8.5263856e-02,  1.7512901e-01, ...,
         7.0279561e-02,  9.6149398e-03, -3.8381062e-02],
       [ 3.1486779e-02, -2.1465714e-01,  6.5056026e-02, ...,
        -1.0497018e-01,  2.5125861e-01, -4.5484155e-03],
       [-4.1211601e-02, -7.7113435e-02,  6.7862630e-02, ...,
         1.7554160e-02,  4.5680344e-02, -2.0435799e-04],
       ...,
       [ 1.8935169e-01, -2.9539612e-01,  2.0197563e-01, ...,
        -1.7756036e-01,  2.1367729e-01, -3.4187708e-02],
       [ 1.9262135e-02, -6.5470494e-02,  6.0493466e-02, ...,
        -4.7835093e-02,  6.1658759e-02,  9.8739423e-02],
       [ 1.8432803e-03, -1.4511240e-02,  5.2576542e-02, ...,
         5.3804882e-02, -8.5479068e-03,  9.4038469e-04]], dtype=float32)

In [157]:
xtest

array([[-0.17682931, -0.22035474,  0.01862513, ..., -0.04123634,
         0.2526633 ,  0.09986784],
       [-0.00667305, -0.12619969,  0.08694158, ...,  0.06372865,
         0.09755841,  0.00682761],
       [ 0.11470237, -0.11503435,  0.08863897, ..., -0.08438297,
         0.14478947,  0.0965043 ],
       ...,
       [-0.03989063, -0.03257908,  0.02932725, ..., -0.00841978,
        -0.02226114, -0.0008958 ],
       [-0.0276941 , -0.04389096,  0.0409498 , ..., -0.08463684,
         0.07066018, -0.01112228],
       [ 0.1879309 , -0.09406086,  0.05426607, ..., -0.01732951,
         0.1617985 ,  0.11900218]], dtype=float32)

In [152]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [153]:
lreg = LogisticRegression(class_weight='balanced', multi_class='auto', max_iter = 1000)

In [154]:
lreg.fit(xtrain, ytrain)

LogisticRegression(class_weight='balanced', max_iter=1000, multi_class='auto')

In [155]:
y_pred = lreg.predict(xtest)

In [156]:
from sklearn.metrics import f1_score, classification_report

print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.60      0.68      1548
           1       0.30      0.46      0.37       432
           2       0.21      0.62      0.31        90
           3       0.43      0.49      0.46       573
           4       0.33      0.45      0.38       317
           5       0.63      0.43      0.51      1606
           6       0.26      0.65      0.37        74
           7       0.43      0.39      0.41       703
           8       0.15      0.54      0.23        84

    accuracy                           0.49      5427
   macro avg       0.39      0.51      0.41      5427
weighted avg       0.56      0.49      0.51      5427

